# Term Project – Preseason Climate Controls on Summer Fire Weather in Manitoba 

This notebook reproduces the original project pipeline:
- **Phase 1:** MAM climate features only  
- **Phase 2:** MAM climate features + ENSO (Niño 3.4 MAM anomaly)
## Reproducibility note
This notebook is a notebook-form version of the original project pipeline.
Minor metric differences relative to the report can occur due to missing-value handling and software/library versions,
but the qualitative conclusions remain unchanged.


In [1]:
from pathlib import Path

DATA_STATIONS = Path("data/stations")
ENSO_FILE     = Path("data/enso/nina34.anom.csv")
FIG_FOLDER    = Path("figures")
FIG_FOLDER.mkdir(exist_ok=True)


In [2]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, ConfusionMatrixDisplay, brier_score_loss
)


In [3]:


def load_all_data(folder):
    files = glob.glob(f"{folder}/*.csv")
    dfs = []

    for f in files:
        df = pd.read_csv(f)
        df["rep_date"] = pd.to_datetime(df["rep_date"], errors="coerce")
        df["year"] = df["rep_date"].dt.year
        df["month"] = df["rep_date"].dt.month
        df["doy"] = df["rep_date"].dt.dayofyear
        dfs.append(df)

    data = pd.concat(dfs, ignore_index=True)
    return data



def compute_mam_features(df):
    mam = df[df["month"].isin([3, 4, 5])].copy()

    feats = mam.groupby(["aes", "year"]).agg(
        T_MAM_mean=("temp", "mean"),
        T_MAM_std=("temp", "std"),
        T_MAM_max=("temp", "max"),
        RH_MAM_mean=("rh", "mean"),
        RH_MAM_min=("rh", "min"),
        W_MAM_mean=("ws", "mean"),
        W_MAM_max=("ws", "max"),
        P_MAM_total=("precip", "sum"),
        P_MAM_wetdays=("precip", lambda x: (x > 0).sum()),
        P_MAM_drydays=("precip", lambda x: (x == 0).sum()),
    ).reset_index()

    return feats



def load_enso_mam(path):
    df = pd.read_csv(path)

    date_col = df.columns[0]
    val_col = df.columns[1]

    df["date"] = pd.to_datetime(df[date_col], errors="coerce")
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    df["nino34"] = pd.to_numeric(df[val_col], errors="coerce")

    df.loc[df["nino34"] == -99.99, "nino34"] = np.nan

    mam = (
        df[df["month"].isin([3, 4, 5])]
        .groupby("year")["nino34"]
        .mean()
        .reset_index()
        .rename(columns={"nino34": "ENSO_MAM"})
    )

    return mam



def build_jja_daily_with_mam(df, mam_features):
    jja = df[df["month"].isin([6, 7, 8])].copy()

    jja = jja.merge(
        mam_features,
        on=["aes", "year"],
        how="left"
    )

    jja = jja.dropna(subset=["fwi"])
    return jja



def build_daily_Xy(jja):
    feature_cols = [
        "doy", "month",
        "T_MAM_mean", "T_MAM_std", "T_MAM_max",
        "RH_MAM_mean", "RH_MAM_min",
        "W_MAM_mean", "W_MAM_max",
        "P_MAM_total", "P_MAM_wetdays", "P_MAM_drydays"
    ]

    if "ENSO_MAM" in jja.columns:
        feature_cols.append("ENSO_MAM")

    X = jja[feature_cols]
    y = (jja["fwi"] >= 20).astype(int)

    mask = jja["year"] < 2010

    X_train, X_test = X[mask], X[~mask]
    y_train, y_test = y[mask], y[~mask]

    return X_train, X_test, y_train, y_test



def evaluate_models(X_train, X_test, y_train, y_test, fig_folder, title_prefix=""):
    models = {
        "Dummy Stratified": DummyClassifier(strategy="stratified", random_state=0),
        "Logistic Regression": Pipeline([
            ("scaler", StandardScaler()),
            ("clf", LogisticRegression(class_weight="balanced", max_iter=1000))
        ]),
        "Random Forest": RandomForestClassifier(n_estimators=200, random_state=0),
        "Gradient Boosting": GradientBoostingClassifier(random_state=0),
    }

    rows = []

    for name, model in models.items():
        model.fit(X_train, y_train)
        prob = model.predict_proba(X_test)[:, 1]
        pred = (prob >= 0.5).astype(int)

        rows.append({
            "Model": name,
            "Accuracy": accuracy_score(y_test, pred),
            "Precision": precision_score(y_test, pred, zero_division=0),
            "Recall": recall_score(y_test, pred, zero_division=0),
            "F1": f1_score(y_test, pred, zero_division=0),
            "Brier": brier_score_loss(y_test, prob),
        })

        cm = confusion_matrix(y_test, pred)
        disp = ConfusionMatrixDisplay(cm)
        disp.plot()
        plt.title(f"{title_prefix}{name}")
        plt.savefig(f"{fig_folder}/{title_prefix}{name}_cm.png")
        plt.close()

    return pd.DataFrame(rows)


In [4]:

data = load_all_data(str(DATA_STATIONS))

print("Loaded daily rows:", len(data))
print("Year range:", int(data["year"].min()), "to", int(data["year"].max()))
print("NaT in rep_date:", data["rep_date"].isna().sum())


mam_features = compute_mam_features(data)
print("Computed MAM features for", len(mam_features), "station-years.")

mam_features.head()


Loaded daily rows: 357573
Year range: 1953 to 2025
NaT in rep_date: 0
Computed MAM features for 2048 station-years.


,aes,year,T_MAM_mean,T_MAM_std,T_MAM_max,RH_MAM_mean,RH_MAM_min,W_MAM_mean,W_MAM_max,P_MAM_total,P_MAM_wetdays,P_MAM_drydays
0,5010480,1960,3.527174,12.104210,27.2,61.717391,21.0,22.850000,48.0,161.23,54,38
1,5010480,1961,5.070652,10.083285,28.3,61.108696,26.0,20.626087,58.0,70.34,33,59
2,5010480,1962,3.584783,10.734631,21.7,68.847826,27.0,22.768478,56.0,125.75,41,51
3,5010480,1963,6.089130,9.710541,27.8,63.717391,30.0,22.636957,56.0,96.82,50,42
4,5010480,1964,4.406522,12.379048,30.0,68.739130,29.0,24.273913,64.0,188.96,45,47


In [5]:
# ---------- PHASE 1: MAM only ----------
jja_p1 = build_jja_daily_with_mam(data, mam_features)

print("Phase 1 JJA rows:", len(jja_p1))
print("Phase 1 years:", int(jja_p1["year"].min()), "to", int(jja_p1["year"].max()))


phase1_feature_cols = [
    "doy", "month",
    "T_MAM_mean", "T_MAM_std", "T_MAM_max",
    "RH_MAM_mean", "RH_MAM_min",
    "W_MAM_mean", "W_MAM_max",
    "P_MAM_total", "P_MAM_wetdays", "P_MAM_drydays"
]

before = len(jja_p1)
jja_p1_clean = jja_p1.dropna(subset=phase1_feature_cols + ["fwi"]).copy()
after = len(jja_p1_clean)
print(f"Phase 1 dropped NaN rows: {before-after:,} (kept {after:,})")

Xtr1, Xte1, ytr1, yte1 = build_daily_Xy(jja_p1_clean)


print("Phase 1 Train size:", len(Xtr1), " Test size:", len(Xte1))
print("Phase 1 Positive rate (test):", float(yte1.mean()))

res1 = evaluate_models(
    Xtr1, Xte1, ytr1, yte1,
    fig_folder=str(FIG_FOLDER),
    title_prefix="PHASE1_"
)

res1


Phase 1 JJA rows: 183618
Phase 1 years: 1953 to 2025
Phase 1 dropped NaN rows: 5,427 (kept 178,191)
Phase 1 Train size: 90678  Test size: 87513
Phase 1 Positive rate (test): 0.18208723275399083


,Model,Accuracy,Precision,Recall,F1,Brier
0,Dummy Stratified,0.722053,0.182403,0.151177,0.165328,0.277947
1,Logistic Regression,0.564408,0.230993,0.597741,0.333217,0.245959
2,Random Forest,0.815022,0.306279,0.012551,0.024114,0.148729
3,Gradient Boosting,0.810108,0.266576,0.024474,0.044833,0.150563


In [6]:
# ---------- PHASE 2: MAM + ENSO ----------


enso_mam = load_enso_mam(str(ENSO_FILE))
print("ENSO_MAM years:", int(enso_mam["year"].min()), "to", int(enso_mam["year"].max()))

mam_features_p2 = mam_features.merge(enso_mam, on="year", how="left")


jja_p2 = build_jja_daily_with_mam(data, mam_features_p2)

print("Phase 2 JJA rows:", len(jja_p2))
print("Phase 2 years:", int(jja_p2["year"].min()), "to", int(jja_p2["year"].max()))


phase2_feature_cols = [
    "doy", "month",
    "T_MAM_mean", "T_MAM_std", "T_MAM_max",
    "RH_MAM_mean", "RH_MAM_min",
    "W_MAM_mean", "W_MAM_max",
    "P_MAM_total", "P_MAM_wetdays", "P_MAM_drydays",
    "ENSO_MAM"
]

before = len(jja_p2)
jja_p2_clean = jja_p2.dropna(subset=phase2_feature_cols + ["fwi"]).copy()
after = len(jja_p2_clean)

print(f"Phase 2 dropped NaN rows: {before-after:,} (kept {after:,})")


Xtr2, Xte2, ytr2, yte2 = build_daily_Xy(jja_p2_clean)

print("Phase 2 Train size:", len(Xtr2), " Test size:", len(Xte2))
print("Phase 2 Positive rate (test):", float(yte2.mean()))


res2 = evaluate_models(
    Xtr2, Xte2, ytr2, yte2,
    fig_folder=str(FIG_FOLDER),
    title_prefix="PHASE2_"
)

res2



ENSO_MAM years: 1948 to 2025
Phase 2 JJA rows: 183618
Phase 2 years: 1953 to 2025
Phase 2 dropped NaN rows: 5,427 (kept 178,191)
Phase 2 Train size: 90678  Test size: 87513
Phase 2 Positive rate (test): 0.18208723275399083


,Model,Accuracy,Precision,Recall,F1,Brier
0,Dummy Stratified,0.722053,0.182403,0.151177,0.165328,0.277947
1,Logistic Regression,0.572829,0.232942,0.587010,0.333529,0.242574
2,Random Forest,0.816359,0.356540,0.010606,0.020598,0.146632
3,Gradient Boosting,0.817513,0.460497,0.012802,0.024911,0.143396


In [7]:
comparison = res1.merge(res2, on="Model", suffixes=("_Phase1", "_Phase2"))
comparison


,Model,Accuracy_Phase1,Precision_Phase1,Recall_Phase1,F1_Phase1,Brier_Phase1,Accuracy_Phase2,Precision_Phase2,Recall_Phase2,F1_Phase2,Brier_Phase2
0,Dummy Stratified,0.722053,0.182403,0.151177,0.165328,0.277947,0.722053,0.182403,0.151177,0.165328,0.277947
1,Logistic Regression,0.564408,0.230993,0.597741,0.333217,0.245959,0.572829,0.232942,0.587010,0.333529,0.242574
2,Random Forest,0.815022,0.306279,0.012551,0.024114,0.148729,0.816359,0.356540,0.010606,0.020598,0.146632
3,Gradient Boosting,0.810108,0.266576,0.024474,0.044833,0.150563,0.817513,0.460497,0.012802,0.024911,0.143396
